In [1]:
!pip install ogb
!pip install memory-profiler

In [2]:
!pip uninstall -y torch torchvision torchaudio torch-geometric torch-sparse torch-scatter torch-cluster torch-spline-conv
!pip uninstall -y torch torchvision torchaudio torch-geometric

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
ERROR: Exception:
Traceback (most recent call last):
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 165, in exc_logging_wrapper
    status = run_func(*args)
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_internal/commands/uninstall.py", line 102, in run
    uninstall_pathset.commit()
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_internal/req/req_uninstall.py", line 420, in commit
    self._moved_paths.commit()
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_internal/req/req_uninstall.py", line 273, in commit
    save_dir.cleanup()
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_internal/utils/temp_dir.py", line 173, in cleanup
    rmtree(self._path)
  File "/home/macierz/s185569/eniu/lib/python3.10/site-packages/pip/_vend

In [3]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp310-cp310-linux_x86_64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.0.2%2Bcu118-cp310-cp310-linux_x86_64.whl (4.4 MB)


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
  Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [5]:
!pip uninstall -y dgl
!pip install dgl-cu117 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [6]:
!apt-get install libcusparse11

E: Nie udało się otworzyć pliku blokady /var/lib/dpkg/lock-frontend - open (13: Brak dostępu)
E: Nie udało się pozyskać blokady interfejsu dpkg (/var/lib/dpkg/lock-frontend), czy użyto uprawnień administratora?


In [7]:
import dgl
import torch

print("DGL version:", dgl.__version__)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Czy GPU jest dostępne:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Używana karta GPU:", torch.cuda.get_device_name(0))

DGL version: 0.9.1post1
PyTorch version: 2.0.1+cu118
CUDA available: True
Czy GPU jest dostępne: True
Używana karta GPU: NVIDIA GeForce RTX 4070 Ti


In [8]:
# Importowanie bibliotek
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from ogb.nodeproppred import PygNodePropPredDataset, NodePropPredDataset
from torch_sparse import SparseTensor
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
from torch_sparse import SparseTensor
import pandas as pd
import csv
import os
import time
import memory_profiler

In [9]:
def load_data(library, matrix_format, dataset_name="ogbn-arxiv"):
    if dataset_name.startswith("ogbn-"):  
        dataset = NodePropPredDataset(name=dataset_name)
        split_idx = dataset.get_idx_split()  # Podziały na train/valid/test
        graph, labels = dataset[0]  # Pobieranie: graf i etykiety

        if library == "PyG":
            import torch_geometric.data as pyg_data
            edge_index = torch.tensor(graph["edge_index"], dtype=torch.long)
            x = torch.tensor(graph["node_feat"], dtype=torch.float)
            y = torch.tensor(labels, dtype=torch.long)

            data = pyg_data.Data(x=x, edge_index=to_undirected(edge_index), y=y)
            data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
            data.train_mask[split_idx["train"]] = True
            data.val_mask[split_idx["valid"]] = True
            data.test_mask[split_idx["test"]] = True

            # Konwersja do wybranego formatu
            data = convert_pyg_format(data, matrix_format)
            return data, dataset_name, dataset

        elif library == "DGL":
            graph_dgl = dgl.graph((graph["edge_index"][0], graph["edge_index"][1]))
            graph_dgl.ndata['feat'] = torch.tensor(graph["node_feat"], dtype=torch.float)
            graph_dgl.ndata['label'] = torch.tensor(labels, dtype=torch.long)
            graph_dgl.ndata['train_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['val_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['test_mask'] = torch.zeros(graph_dgl.num_nodes(), dtype=torch.bool)
            graph_dgl.ndata['train_mask'][split_idx["train"]] = True
            graph_dgl.ndata['val_mask'][split_idx["valid"]] = True
            graph_dgl.ndata['test_mask'][split_idx["test"]] = True

            # Konwersja do wybranego formatu
            graph_dgl = convert_dgl_format(graph_dgl, matrix_format)
            return graph_dgl, dataset_name, dataset

    else:
        # Obsługa innych zbiorów danych
        return 1,2,3

In [31]:
# Funkcja obsługująca formaty macierzy dla PyTorch Geometric (PyG)
def convert_pyg_format(data, matrix_format):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = "cpu"
    if matrix_format == "COO":
        # Dane już w formacie COO (edge_index)
        return data
    elif matrix_format == "CSR":
        # Konwersja do CSR (SparseTensor)
        data.adj_t = SparseTensor(row=data.edge_index[0], col=data.edge_index[1], value=None).to(device)
        return data
    elif matrix_format == "CSC":
        # Konwersja do CSR, a potem transpozycja do CSC
        data.adj_t = SparseTensor(row=data.edge_index[0], col=data.edge_index[1], value=None).to(device)
        data.adj_t = data.adj_t.t()  # Użycie t() zamiast transpose()
        return data
    else:
        raise ValueError("Nieznany format macierzy dla PyG: {}".format(matrix_format))

# Funkcja obsługująca formaty macierzy dla DGL
def convert_dgl_format(graph, matrix_format):
    if matrix_format == "COO":
        # DGL domyślnie obsługuje COO, nie trzeba konwertować
        return graph
    elif matrix_format == "CSR":
        # Konwersja do formatu CSR
        return dgl.to_block(dgl.to_simple(graph))  # Konwersja na CSR
    elif matrix_format == "CSC":
        graph = dgl.add_self_loop(graph)  # DGL nie wspiera natywnie CSC, dodanie self-loop
        return graph
    else:
        raise ValueError("Nieznany format macierzy dla DGL: {}".format(matrix_format))


In [32]:
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from torch_sparse import SparseTensor
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, adjs):
        edge_index, _, size = adjs[0]  # Pierwsza warstwa
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        edge_index, _, size = adjs[1]  # Druga warstwa
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, x, adjs):
        edge_index, _, size = adjs[0]
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        edge_index, _, size = adjs[1]
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)

    def forward(self, x, adjs):
        edge_index, _, size = adjs[0]
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        edge_index, _, size = adjs[1]
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [33]:
import torch.optim as optim
import torch.nn.functional as F
import gc

In [34]:
import torch
from torch_geometric.loader import NeighborSampler
import torch.nn.functional as F
import csv
import os
import time

def train_with_batching(model, data, num_epochs, batch_size=32):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # Zmniejszanie LR

    train_loader = NeighborSampler(
        edge_index=data.edge_index,
        node_idx=data.train_mask,
        sizes=[5, 5],
        batch_size=batch_size,
        shuffle=True
    )

    model.train()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = "cpu"
    model.to(device)

    # Sprawdzanie danych wejściowych
    print(f"Feature shape: {data.x.shape}, Labels shape: {data.y.shape}, Labels range: {data.y.min()} - {data.y.max()}")
    print(f"Train mask size: {data.train_mask.sum()}, Test mask size: {data.test_mask.sum()}")

    if data.train_mask.sum() == 0:
        raise ValueError("Train mask is empty. Check the dataset.")

    total_time = 0

    for epoch in range(num_epochs):
        epoch_loss = 0
        start_time = time.time()

        for batch_size, n_id, adjs in train_loader:
            adjs = [adj.to(device) for adj in adjs]
            x_batch = data.x[n_id].to(device)
            y_batch = data.y[n_id[:batch_size]].to(device)

            if y_batch.dim() > 1:
                y_batch = y_batch.squeeze(dim=-1)

            optimizer.zero_grad()
            out = model(x_batch, adjs)
            out = out[:batch_size]

            loss = F.cross_entropy(out, y_batch)
            loss.backward()
            optimizer.step()

            # Debugowanie batcha
            #print(f"Batch output range: {out.min().item()} - {out.max().item()}")
            #print(f"Batch Loss: {loss.item():.4f}")

            epoch_loss += loss.item()

        scheduler.step()  # Aktualizacja learning rate
        end_time = time.time()
        epoch_time = end_time - start_time
        total_time += epoch_time

        #Epoch log
        print(f"Epoch {epoch + 1}/{num_epochs}, Time: {epoch_time:.2f}s")
        print(f"Learning rate after epoch {epoch + 1}: {scheduler.get_last_lr()[0]:.6f}")

    return total_time, 0


from torch_geometric.loader import NeighborSampler

def evaluate_model(model, data):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = "cpu"

    test_loader = NeighborSampler(
        edge_index=data.edge_index,
        node_idx=data.test_mask,
        sizes=[5, 5], 
        batch_size=32,
        shuffle=False
    )

    correct = 0
    total = 0
    inference_time = 0

    with torch.no_grad():
        for batch_size, n_id, adjs in test_loader:
            adjs = [adj.to(device) for adj in adjs]
            x_batch = data.x[n_id].to(device)
            y_batch = data.y[n_id[:batch_size]].to(device)

            if y_batch.dim() > 1:
                y_batch = y_batch.squeeze(dim=-1)
                
            start_time = time.time()
            out = model(x_batch, adjs)
            end_time = time.time()
            inference_time += (end_time - start_time)
            out = out[:batch_size]  # Przycinanie wyjść modelu

            pred = out.argmax(dim=1)

            # Debugowanie
            #print(f"Predykcje: {pred[:10]}, Prawdziwe: {y_batch[:10]}")

            correct += (pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(f"Correct: {correct}, Total: {total}, Accuracy: {accuracy:.4f}")
    return accuracy, inference_time




In [35]:
def run_experiments():
    models = {
        'GCN': GCN,
        'GraphSAGE': GraphSAGE,
        'GAT': GAT
    }
    formats = ["COO", "CSR", "CSC"]
    libraries = ["PyG"]  

    results = []
    file_exists = os.path.isfile('experiment_results_updated_g.csv')
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #device = "cpu"

    with open('experiment_results_updated_g.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Model", "Library", "Format", "Dataset", "E2E Time (s)", "Training Time (s)", "Inference Time (s)", "Validation Accuracy", "Device"])
            
        for model_name, model_class in models.items():
            for lib in libraries:
                for fmt in formats:
                    start_e2e_time = time.time()  # Rozpoczęcie pomiaru E2E

                    data, dataset_name, dataset = load_data(lib, fmt)
                    print(f"Running experiment for {model_name} with {lib} using {fmt} format on {dataset_name}.")

                    if lib == "PyG":
                        model = model_class(input_dim=data.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
                        # Batchowane trenowanie
                        train_time, mem_usage = train_with_batching(model, data, num_epochs=10, batch_size=32) #TRENOWANIE
                    elif lib == "DGL":
                        feat = data.ndata['feat']
                        label = data.ndata['label']

                        # Sprawdzenie, czy feat to dict
                        if isinstance(feat, dict):
                            feat = feat['_N']  
                        if isinstance(label, dict):
                            label = label['_N']  

                        model = model_class(input_dim=feat.shape[1], hidden_dim=16, output_dim=label.max().item() + 1)
                        train_time, mem_usage = train(model, data)  # Klasyczne trenowanie dla DGL

                    accuracy, inference_time = evaluate_model(model, data)  # Ewaluacja modelu
                    end_e2e_time = time.time()  # Koniec pomiaru E2E

                    e2e_time = end_e2e_time - start_e2e_time
                    print(f"E2E time: {e2e_time:.2f}s")

                    writer.writerow([model_name, lib, fmt, dataset_name, e2e_time, train_time, inference_time, accuracy, device])
                    results.append([model_name, lib, fmt, dataset_name, e2e_time, train_time, inference_time, accuracy, device])

                    print(f"Finished {model_name} with {lib} using {fmt} format on {dataset_name}.")

    return results

In [36]:
results = run_experiments()

Running experiment for GCN with PyG using COO format on ogbn-arxiv.
Feature shape: torch.Size([169343, 128]), Labels shape: torch.Size([169343, 1]), Labels range: 0 - 39
Train mask size: 90941, Test mask size: 48603
Epoch 1/10, Loss: 1155.8169, Time: 4.27s
Learning rate after epoch 1: 0.010000
Epoch 2/10, Loss: 986.0138, Time: 4.44s
Learning rate after epoch 2: 0.010000
Epoch 3/10, Loss: 970.6119, Time: 4.36s
Learning rate after epoch 3: 0.010000
Epoch 4/10, Loss: 962.4547, Time: 4.35s
Learning rate after epoch 4: 0.010000
Epoch 5/10, Loss: 956.4280, Time: 4.40s
Learning rate after epoch 5: 0.005000
Epoch 6/10, Loss: 939.8260, Time: 4.46s
Learning rate after epoch 6: 0.005000
Epoch 7/10, Loss: 938.4294, Time: 4.13s
Learning rate after epoch 7: 0.005000
Epoch 8/10, Loss: 936.3611, Time: 4.44s
Learning rate after epoch 8: 0.005000
Epoch 9/10, Loss: 935.5768, Time: 4.39s
Learning rate after epoch 9: 0.005000
Epoch 10/10, Loss: 935.3048, Time: 4.32s
Learning rate after epoch 10: 0.002500
C

In [1]:
#results = run_experiments()

In [2]:
#results = run_experiments()